In [3]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

import openpyxl
import pandas as pd

import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/home/dwna/projects/domain_class')
from src.features.build_features import BuildFeatures


In [5]:
pd.set_option('display.max.colwidth', None,
              'display.max_columns', None,
)



In [6]:

file_path = "quality_evaluation_data.xlsx"

df_e = pd.read_excel(file_path, sheet_name='error_samples')
df_n = pd.read_excel(file_path, sheet_name='normal_samples')

err_columns = ['table', 'col_nm', 'domain', 'rule', 'total_cnt', 'err_cnt', 'err_rt', 'err_ext_sql',
       'err_data']
nor_columns = ['table', 'col_nm', 'datatype', 'rule', 'domain', 'verify_type', 'err_free_data', 'act_date',
       'normal_data', 'others']
df_e.columns = err_columns
df_n.columns = nor_columns



columns = ['table', 'col_nm', 'domain', 'normal_data', 'err_data', 'datatype', 'rule',  'total_cnt', 'err_cnt', 'err_rt']
df = df_n.merge(df_e, 'left', on=['table','col_nm'],suffixes=('','_y')) 

df = (df[columns]
        [~df.normal_data.isna()]
        .assign(normal_data = df.normal_data.astype(str).str.split(',').apply(lambda x:[] if x ==['nan'] else x), 
                err_data = df.err_data.astype(str).str.split(',').apply(lambda x:[] if x ==['nan'] else x)
               )
)



In [7]:


import random

def generate_combined_set(normal_list, abnormal_list, abnormal_probability, length):
    # Early return if both lists are empty
    if not normal_list and not abnormal_list:
        return []

    # Adjust counts if one of the lists is empty
    abnormal_count = int(length * abnormal_probability) if abnormal_list else 0
    normal_count = length - abnormal_count

    # Generate lists based on the counts
    chosen_abnormal = random.choices(abnormal_list, k=abnormal_count) if abnormal_list else []
    chosen_normal = random.choices(normal_list, k=normal_count) if normal_list else []

    # Combine and shuffle the final list
    final_list = chosen_normal + chosen_abnormal
    random.shuffle(final_list)

    return final_list


def profile_data(df: pd.DataFrame, abnormal_rate: float, length: int)->pd.DataFrame:
    """
    Profile the data from a DataFrame using the BuildFeatures class.
    """
    profiles = []

    for row in df.itertuples(index=False):
        combined = generate_combined_set(row.normal_data, row.err_data, abnormal_rate, length)
        series = pd.Series(combined, name=row.col_nm)

        profile = BuildFeatures(series, domain=row.domain).profiling_patterns()
        profiles.append(profile)
    
    return pd.DataFrame(profiles)


# profiles = profile_data(df, abnormal_rate=0.01, length=1000)


from joblib import Parallel, delayed
import dask.dataframe as dd

def profile_row(row, abnormal_rate, length):
    warnings.filterwarnings('ignore')  # multiprocessing시 warning 출력 안되게 하기위해 설정함.
    combined = generate_combined_set(row.normal_data, row.err_data, abnormal_rate, length)
    series = pd.Series(combined, name=row.col_nm)
    profile = BuildFeatures(series, domain=row.domain).profiling_patterns()
    return profile

def profile_data_joblib(df: pd.DataFrame, abnormal_rate: float, length: int, num_cores:int=-1) -> pd.DataFrame:
    # Prepare the data for parallel processing
    results = Parallel(n_jobs=num_cores)(delayed(profile_row)(row,abnormal_rate, length) 
                                  for row in df.itertuples(index=False))
    return pd.DataFrame(results)

def profile_data_dask(df: pd.DataFrame, abnormal_rate: float, length: int, number_of_partitions:int =10) -> pd.DataFrame:
    # Prepare the data for parallel processing
    ddf = dd.from_pandas(df, npartitions=number_of_partitions)
    result = ddf.map_partitions(lambda partition: partition.apply(lambda row: profile_row(row,  abnormal_rate, length), axis=1)) \
                .compute(scheduler='processes')

    return result


In [8]:


profiles = profile_data(df, abnormal_rate=0.01, length=1000)


In [9]:

df_profiled = profile_data_dask(df, abnormal_rate=0.01, length=1000, number_of_partitions=5)


In [10]:

profiles_job = profile_data_joblib(df, abnormal_rate=0.01, length=1000)

In [11]:
profiles_job

,col_name,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,part_minus,len_purity,value_nunique,value_distr,datatype,BUNHO,NALJJA,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC,domain
0,ENTN,0.0,0.010,0.010,1.000000,0.0,0.0,1.000000,0.00,1.000000,0.0,0.0,0.980200,6,2.378638,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
1,GRADE,0.0,1.000,1.000,1.000000,0.0,0.0,1.000000,0.00,1.000000,0.0,0.0,1.000000,2,0.999766,object,0,0,0,0,0,0,0,0,0,0,0,True,코드
2,PWD,0.0,0.779,0.779,0.997000,0.0,0.0,0.994000,0.01,0.994000,0.0,0.0,1.000000,12,2.402680,object,0,0,0,0,0,0,0,0,0,0,0,True,번호
3,USNA,0.0,0.000,0.000,1.000000,0.0,0.0,0.000000,1.00,0.000000,0.0,0.0,1.000000,5,2.316053,object,0,0,0,0,0,0,0,0,0,0,0,True,NaN
4,APPR_GIVE_DATE,1.0,0.000,0.000,0.000000,0.0,0.0,1.000000,0.00,1.000000,0.0,0.0,1.000000,5,2.321663,object,0,0,0,0,0,0,0,0,0,0,0,True,날짜
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,EMP_NO,0.0,0.750,0.750,0.751276,0.0,0.0,0.751276,0.00,0.751276,0.0,0.0,0.608591,11,2.107733,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
1955,DEPT_NM,0.0,0.000,0.000,1.000000,0.0,0.0,0.000000,1.00,0.000000,0.0,0.0,0.597191,4,1.996275,object,0,0,True,0,0,0,0,0,0,0,0,0,NaN
1956,DEPT_TEL,0.0,0.000,0.000,0.745478,0.0,0.0,0.745478,0.00,0.745478,0.0,0.0,0.371215,4,1.999411,object,0,0,0,0,0,0,0,0,0,0,0,True,번호
1957,CMPT_AUTH,0.0,0.809,0.809,1.000000,0.0,0.0,1.000000,0.00,1.000000,0.0,0.0,1.000000,5,2.321023,object,0,0,0,0,0,0,0,0,0,0,0,True,코드


# FYI

In [69]:
import openpyxl
import pandas as pd

file_path = "quality_evaluation_data.xlsx"
df_e = pd.read_excel(file_path, sheet_name='error_samples')
df_n = pd.read_excel(file_path, sheet_name='normal_samples')
err_columns = ['table', 'col_nm', 'domain', 'rule', 'total_cnt', 'err_cnt', 'err_rt', 'err_ext_sql',
       'err_data']
nor_columns = ['table', 'col_nm', 'datatype', 'rule', 'domain', 'verify_type', 'err_free_data', 'act_date',
       'nornal_data', 'others']
df_e.columns = err_columns
df_n.columns = nor_columns
df = df_n.merge(df_e, 'left', on=['table','col_nm'])
df = df[['table', 'col_nm', 'domain_x', 'nornal_data', 'err_data', 'datatype', 'rule_x',  'total_cnt', 'err_cnt', 'err_rt']]
df =df.rename(columns={'nornal_data':'normal_data', 'rule_x':'rule', 'domain_x':'domain'})
df = df[~df.normal_data.isna()]
df = df.assign(normal_data = df.normal_data.str.split(',').astype('str'),
             err_data = df.err_data.str.split(',').astype('str'),
)

In [56]:
# build_feature.py
import pandas as pd
import numpy as np
import re
from pandas import Series


class BuildFeatures():
    """
    ## 전처리
    - None, NULL 등의 갯수/비율 체크 및 삭제
    ##전체패턴
    - 날짜 형식 비율 
    - 숫자 비율(실수, 정수, 00012같은 것은 걸려낼 것)
    - 정수 비율
    - 번호패턴 비율
    - 문자번호패턴 비율 
    - 이메일 비율
    - URL 비율
    ##부분패턴
    - 숫자포함
    - 문자포함(한글영문)
    - discriminator포함
    - masking포함
    - 음수포함
    ## 데이터 속성
    - 데이터길이 Purity : 0~1, 1에 가까울수록 고정길이, (엔트로피를 사용해도 될 듯)
    - 데이터 Distinct 수
    - 데이터 엔트로피 : distinct 값의 분포. Noise로 인한 distinct가 크면 의미가 있는데, 계산량이 많이 들 수 있다.
    ##메타데이터
    - 컬럼도메인: 컬럼명으로부터 도메인 접미사 추출하여 해당도메인 onehot처리(도메인별 접미사 구성)
    - 데이터 타입
    
    # usage
        data = {
            'sample_column': [
                '2023-01-01', '100', '300.5', 'test@example.com', 'http://example.com', 
                '2023-02-01', '200', '400.5', 'hello@world.com', 'https://world.com',
                '2023-03-01', 'NULL', None, 'NaN', '2023-04-01'
            ]
        }
        # 판다스 데이터프레임 생성
        df = pd.DataFrame(data)
        # BuildFeatures 클래스 인스턴스화
        bf = BuildFeatures(df['sample_column'], 'SAMPLE_sz')
        # 프로파일링 패턴 호출
        profile = bf.profiling_patterns()
        print(profile)    
            
    """
    YYYY = r"(19|20)\d{2}"
    MM = r"(0[1-9]|1[012])"
    DD = r"(0[1-9]|[12][0-9]|3[01])"
    TM = r"\s+([01][0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?(\s+(PM|AM))?"
    patterns = {
        "date_time": fr"^({YYYY}[-./]{MM}[-./]{DD}|{MM}[-./]{DD}[-./]{YYYY}|{DD}[-./]{MM}[-./]{YYYY})({TM})?$",
        "number": r"^(?!0\d)\d+([.]\d*)?$",
        "integer": r"^(?!0\d)\d+$",
        "bunho": r"^[A-Za-z0-9\uAC00-\uD7A3]+([-./:][A-Za-z0-9\uAC00-\uD7A3]+)*$",
        "email": r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$",
        "url": r"^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$",
        "part_num": r"\d+",
        "part_text": r"[A-Za-z\uAC00-\uD7A3]+",
        "part_discriminator": r"[./-:]",
        "part_mask": r"[#*]{3,}",
        "part_minus": r"^-\d",
    }

    #  도메인 접미사 패턴 그룹 정의 및 결합
    suffix_patterns = {
        "BUNHO": "(?P<BUNHO>(NO|SN|ZIP|TKN|VIN|ENTN)$)",
        "NALJJA": "(?P<NALJJA>(DT|YMD|YM|YR|SYR|MM|DAY)$)",
        "MYEONG": "(?P<MYEONG>(NM|TTL)$)",
        "JUSO": "(?P<JUSO>(ADDR)$)",
        "YEOBU": "(?P<YEOBU>YN$)",
        "CODE": "(?P<CODE>CD$)",
        "ID": "(?P<ID>ID$)",
        "SURYANG": "(?P<SURYANG>(LOT|DONT|GRD|LVL|GFA|AREA|PRG|SCR|CNT|LEN|SZ)$)",
        "GEUMAEK": "(?P<GEUMAEK>(AMT|FEE|UNTPRC)$)",
        "NAEYOUNG": "(?P<NAEYOUNG>CN$)",
        "YUL": "(?P<YUL>RT$)",
    }
    combined_pattern = re.compile("|".join(suffix_patterns.values()),  re.IGNORECASE)
    
    def __init__(self, series:Series, col_name:str=None):
        if not isinstance(series, pd.Series):
            raise ValueError("series must be a pandas Series")        
        
        na_rate = series.isna().mean()
        null_rate = series.str.strip().isin(['NULL', 'NaN']).mean()
        
        self.null_rate = na_rate + null_rate
        self.series = series[~series.isin(['NULL', 'NaN'])].dropna()
        self.datatype = self.series.dtype.name  
        self.col_name = col_name if col_name else self.series.name

        # self.patterns = patterns
        
    def rate_matching_pattern(self, pattern:str)-> float:
        # series = self.series.dropna()
        return self.series.str.contains(pattern, regex=True).mean()
    

    def get_length_purity(self)-> float:
        # series = self.series.dropna().astype(str)
        ratio = self.series.str.len().value_counts(normalize=True)
        length_purity = (ratio * ratio).sum()
        return length_purity
    
    def get_value_nunique(self):
        return self.series.nunique()
        # self.series.value_counts()
        
    def get_value_distr(self):
        ratio = self.series.value_counts(normalize=True)
        entropy = (-ratio * np.log2(ratio)).sum()
        return entropy
    
    def find_suffix_domain(self):
        """
        컬럼명에서 도메인분류어 패턴을 추출하여 어떤 도메인그룹의 suffix에 해당하는지 검출
        :return: 매칭된 그룹 이름 또는 '해당 사항 없음'
        """
        word = self.col_name
        if not isinstance(word, str) or not word:
            return "유효하지 않은 입력"
        
        match = self.combined_pattern.search(word)
        if match:
            matched_groups = [name for name, value in match.groupdict().items() if value]
            return matched_groups if matched_groups else "ETC"
        else:
            return "ETC"
           
    def one_hot_encode(self, domain:str, categories: list) -> dict:
       """
       주어진 pandas Series를 one-hot 인코딩으로 변환합니다.
    
       :param series: 인코딩할 pandas Series 객체
       :param categories: one-hot 인코딩을 위한 카테고리 리스트
       :return: 카테고리 값을 키로 하고 one-hot 인코딩된 pd.Series를 값으로 하는 딕셔너리
       """
       if not isinstance(categories, list):
           categories = list(categories)
       categories = categories +['ETC'] # categories에 들어가지 않는 것은 ETC로 추가로 분류한다.
       one_hot_encoded = pd.get_dummies(domain, prefix='', prefix_sep='').reindex(columns=categories, fill_value=0)
       return one_hot_encoded.iloc[0].to_dict() 

    def profiling_patterns(self)-> Series:
        features = {}
        for key, pattern in self.patterns.items():
            features[key] = self.rate_matching_pattern(pattern)
        features['len_purity'] = self.get_length_purity()
        features['value_nunique'] = self.get_value_nunique()
        features['value_distr'] = self.get_value_distr()
        features['datatype'] = self.datatype
        suffix_domain = self.find_suffix_domain()
        one_hot_encoded = self.one_hot_encode(suffix_domain, self.suffix_patterns.keys()) 
        features = {**features, **one_hot_encoded}
        
        return pd.Series(features,name = self.col_name )
    


In [ ]:
### use_case
data = {
    'sample_column': [
        '2023-01-01', '100', '300.5', 'test@example.com', 'http://example.com', 
        '2023-02-01', '200', '400.5', 'hello@world.com', 'https://world.com',
        '2023-03-01', 'NULL', None, 'NaN', '2023-04-01'
    ]
}


data = {

    'ENTN': [
        '200905830','201785062','080146287','200100957','201810936'
    ]
}

# 판다스 데이터프레임 생성
df = pd.DataFrame(data)
# BuildFeatures 클래스 인스턴스화
bf = BuildFeatures(df['ENTN'], 'ENTN')
# 프로파일링 패턴 호출
profile = bf.profiling_patterns()
print(profile)    


In [83]:
df.normal_data[0]

"['080068544', '080134675', '080135855', '080139164', '080147596']"

In [84]:
def str2series(str_list):
    return pd.Series(eval(str_list))

In [87]:
df.columns

Index(['table', 'col_nm', 'domain', 'normal_data', 'err_data', 'datatype',
       'rule', 'total_cnt', 'err_cnt', 'err_rt'],
      dtype='object')

In [88]:
profiles =[]
for row in df[:10].itertuples():
    series = str2series(row.normal_data)
    series.name = row.col_nm
    profile = BuildFeatures(series).profiling_patterns()
    profiles.append(profile)
    

/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is i

In [91]:
type(profile)

pandas.core.series.Series

In [96]:
pd.DataFrame(profiles)

,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,...,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC
ENTN,0.0,0.00,0.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
GRADE,0.0,1.00,1.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
PWD,0.0,0.80,0.80,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
USNA,0.0,0.00,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,True
APPR_GIVE_DATE,1.0,0.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
OPER_GIVE_DATE,1.0,0.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
OPER_GIVE_NAME,0.0,0.00,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,True
BUSN_REGN,0.0,0.75,0.75,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
ENTN,0.0,0.80,0.80,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
GRADE,0.0,1.00,1.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True


In [ ]:
df.normal_data

0                                                         080068544,080134675,080135855,080139164,080147596
1                                                                                                       1,2
2                                                                                  0000,1234,1111,7168,1004
3                                                                                       담당자,김주희,정경민,김가영,이정희
4       2018-10-16 17:04:54,2019-07-15 14:10:07,2008-09-25 13:41:09,2008-09-26 15:21:20,2008-10-13 17:27:52
                                                       ...                                                 
2237                                                                      NULL,20180068,-,19910022,19960081
2239                                                                         NULL,자원순환지원부,청소행정과,환경관리과,자원순환과
2240                                                             NULL,1644-0007,02-3153-0514,-,031-590-0620
2241                        

In [61]:
pd.set_option('display.max.colwidth', None)
df.query('domain == "번호"')['normal_data']

0                             ['080068544', '080134675', '080135855', '080139164', '080147596']
2                                                      ['0000', '1234', '1111', '7168', '1004']
7                              ['NULL', '0000000000', '1208200052', '3068200471', '6138300022']
8                             ['131037196', '080146287', '200022389', '200100957', '202211277']
10                ['031-634-1762', '0629538902', '0319723084', '043-750-7395', '010-2542-4407']
                                                 ...                                           
2216                                                              ['NULL', '2', '3', '6', '10']
2231    ['210.99.81.253', '210.99.81.254', '210.99.81.251', '210.104.107.34', '210.104.107.50']
2233                          ['200305241', '200309912', '200311019', '200412103', '200418018']
2237                                          ['NULL', '20180068', '-', '19910022', '19960081']
2240                                 ['N

In [67]:
df.normal_data[0]

"['080068544', '080134675', '080135855', '080139164', '080147596']"